# Notebook principal

## Resumo

Este notebook contém pré-tratamento de dados, otimização de hiperparâmetros e estratégias de tratamento de dados, treino e teste para cada modelo e análise de resultados. O dataset estudado possui target binário e, com isso em mente, são treinados os seguintes algoritmos de aprendizado de máquina: baseline, regressão logística, K-nn classificador, congresso de K-nn classificador, floresta aleatória classificadora, e máquina de vetores de suporte (SVM) classificador. 

Para cada modelo tem-se um optuna que testa estratégias de tratamento de dados (redução de dimensionalidade, seleção de atributos e normalização) e pelo menos 100 conjuntos de hiperparâmetros para cada algoritmo. A função objetivo do optuna retorna o RMSE que é calculado como a média aparada dos valores obtidos de uma validação cruzada 10-Fold.

## Introdução

## Coleta de dados

In [53]:
import pandas as pd

# Carrega os dados do arquivo
df = pd.read_csv('earthquake_data_tsunami.csv')

# Mostra 5 primeiras linhas do dataset
df.head()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.000,-9.7963,159.596,2022,11,1
1,6.9,4,4,735,99,2.229,34.0,25.000,-4.9559,100.738,2022,11,0
2,7.0,3,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,2022,11,1
3,7.3,5,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,2022,11,1
4,6.6,0,2,670,131,4.998,27.0,624.464,-25.5948,178.278,2022,11,1


In [50]:
print(f'O tamanho do df antes do pré-tratamento de dados é: {len(df)}')

O tamanho do df antes do pré-tratamento de dados é: 782


A coluna cdi apresenta valores 0 que indicam a não disponibilidade de informação, portanto, são dados que devem ser removidos. O código abaixo faz uma lista com todos os registros que possuem cdi 0 e contabiliza.

In [51]:
# Define índices com cdi=0
indices = [i for i in df.index if df['cdi'][i]==0]
print(f'O número de índices com cdi igual a zero é: {len(indices)}')

O número de índices com cdi igual a zero é: 212


Como cdi não traz muita informação considerando a presença do mmi, não vale a pena perder 212 registros de terremotos para preservar a coluna de cdi. Ela será removida durante o pré-tratamento de dados.

## Pré-tratamento de dados

In [52]:
from sklearn.feature_selection import VarianceThreshold

# Retira colunas que não serão usadas
df = df.drop(['Year', 'Month', 'cdi'], axis=1)

# Remove índices com NaN
df = df.dropna()

# Retira colunas com variância zero
seletor = VarianceThreshold()
seletor.fit(df)
df_ = seletor.transform(df) # df vira np.ndarray

# Transforma em DataFrame
df = pd.DataFrame(df_, columns=[*df.columns])

print(f'O tamanho do df após pré-tratamento de dados é: {len(df)}')
print()

O tamanho do df após pré-tratamento de dados é: 782



Não foi perdido nenhum resgistro de terremoto, o que significa que não havia dados com NaN.

In [47]:
df.head()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,tsunami
0,7.0,8.0,7.0,768.0,117.0,0.509,17.0,14.0,-9.7963,159.596,1.0
1,6.9,4.0,4.0,735.0,99.0,2.229,34.0,25.0,-4.9559,100.738,0.0
2,7.0,3.0,3.0,755.0,147.0,3.125,18.0,579.0,-20.0508,-178.346,1.0
3,7.3,5.0,5.0,833.0,149.0,1.865,21.0,37.0,-19.2918,-172.129,1.0
4,7.0,4.0,3.0,755.0,142.0,4.578,26.0,660.0,-26.0442,178.381,1.0


Não foi perdida nenhuma coluna, o que significa que nenhuma possui variância zero.

In [30]:
# Define colunas de atributos e target
TARGET = ['tsunami']
ATRIBUTOS = [*df.columns] # Adiciona todas as colunas
ATRIBUTOS.remove(*TARGET) # Retira colunas target

print(f'Colunas atributos: {ATRIBUTOS}')
print()
print(f'Coluna target: {TARGET}')

Colunas atributos: ['magnitude', 'cdi', 'mmi', 'sig', 'nst', 'dmin', 'gap', 'depth', 'latitude', 'longitude']

Coluna target: ['tsunami']


In [24]:
from sklearn.model_selection import train_test_split

SEMENTE_ALEATORIA = 9

# Faz split em treino e teste

# Parâmetro para split
TAMANHO_TESTE = 0.1

# Define índices de split
indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

# Separa o DataFrame em treino e teste
df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

# Separa os DataFrames em atributos e target

# Treino
X_treino = df_treino.reindex(ATRIBUTOS, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

# Teste
X_teste = df_teste.reindex(ATRIBUTOS, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()